In [ ]:
from pystac_client import Client
import xarray as xr
import datetime as dt
import rioxarray as rio
import geopandas as gpd
from rasterio.enums import Resampling
from pathlib import Path

In [ ]:
#helper function to search sentinel-2 images
def get_sentinel2_collections(start_date,end_date,intersects_geom,max_cloud_cover=50,max_items=10):
    client = Client.open("https://earth-search.aws.element84.com/v1")
    search = client.search(
        collections=["sentinel-2-l2a"],
        intersects=intersects_geom,
        datetime=f"{start_date}/{end_date}",
        query=[f"eo:cloud_cover<{max_cloud_cover}"],
        max_items=max_items
        )
    
    if nr_matches:=search.matched():
        print(f"Found {nr_matches} image(s)")
        return search.item_collection()
        

In [ ]:
#load aoi from parquet
aoi_parquet = Path("data/cleaned/clients/dummy_plantations.parquet")
aoi_crs = aoi_parquet.crs.to_epsg()
aoi_gdf = gpd.read_parquet(aoi_parquet)
aoi_gdf['geometry'] = aoi_gdf['geometry'].explode(index_parts=True).reset_index(drop=True) #fix for multipolygons
aoi_gdf['geometry'] = aoi_gdf['geometry'].minimum_bounding_circle()


In [ ]:
#sentinel2 search params
lag0days_dt = dt.datetime.today()
start_date = (lag0days_dt - dt.timedelta(days=7)).strftime('%Y-%m-%d')
end_date = lag0days_dt.strftime('%Y-%m-%d')
intersects_geom_wgs84 = aoi_gdf.to_crs(4236).dissolve().concave_hull().__geo_interface__['features'][0]['geometry']

In [ ]:
#create and write NDVI and RGB images
collections = get_sentinel2_collections(start_date,end_date,intersects_geom_wgs84)
for obj in collections:

    #create geojson for clipping images
    epsg_crs = obj.properties['proj:epsg']
    clip_geojson = [aoi_gdf.to_crs(epsg_crs).dissolve().__geo_interface__['features'][0]['geometry']]
    
    #clip, stack and reproject images
    imgs = []
    for item,asset in obj.assets.items():
        if item in ['red', 'green', 'blue','nir']:
            img = (rio.open_rasterio(asset.href)
                   .isel(band=0)
                   .drop_vars(['band'])
                   .to_dataset(name=item)
                   .rio.clip(clip_geojson, from_disk=True, all_touched=True))
            imgs.append(img)
    stacked_img = xr.combine_by_coords(imgs).rio.reproject(dst_crs=f"epsg:{aoi_crs}", resolution=10, resampling=Resampling.bilinear)  
    
    #compose new images
    ndvi_img = ((stacked_img['nir']  - stacked_img['red'] )/(stacked_img['nir']  +  stacked_img['red'] )).to_dataset(name='ndvi').astype('float32')
    rgb_img = stacked_img[['red','green','blue']]

    #export new images
    ndvi_img.rio.to_raster(f'{obj.id}_ndvi.tif', driver='COG')
    rgb_img.rio.to_raster(f'{obj.id}_rgb.tif', driver='COG')